<h1>擷取與視覺化股票資料</h1>
<h2>描述</h2>


從資料集中提取基本資料並顯示它是資料科學的必要組成部分；因此個人可以根據數據做出正確的決定。在此作業中，您將提取一些股票數據，然後將這些數據顯示在圖表中。

<h2>目錄</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>定義一個產生圖表的函數</li>
        <li>使用yfinance擷取股票數據</li>
        <li>使用Webscraping提取特斯拉收入數據</li>
        <li>使用yfinance擷取股票數據</li>
        <li>使用Webscraping提取GME收入數據</li>
        <li>繪製 Tesla 股票圖</li>
        <li>繪製 GameStop 股票圖</li>
    </ul>
<p>

</div>

<hr>


In [1]:
!pip install yfinance
!pip install bs4
!pip install nbformat

In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 定義繪圖函數


在本節中，我們定義函數 `make_graph`。 您不必知道該函數如何工作，您應該只關心輸入。 它需要一個包含股票資料的資料框（資料框必須包含日期和關閉列）、一個包含收入資料的資料框（資料框必須包含日期和收入列）以及股票名稱。


In [26]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## 使用yfinance擷取股票資料


使用 `Ticker` 函數輸入我們要提取資料的股票的股票代碼，以建立股票物件。該股為 `Tesla`，股票代號為 `TSLA`。


In [27]:
Tesla = yf.Ticker("TSLA")

使用股票代碼物件和函數`history`提取股票資訊並將其保存在名為`tesla_data`的資料框中。將 `period` 參數設為 `max`，以便我們獲得最長時間的資訊。


In [28]:
tsla_data = Tesla.history(period="max")

TSLA: No data found for this date range, symbol may be delisted


使用 **tesla_data DataFrame** 上的 `reset_index(inplace=True)` 函數重設索引，並顯示 `tesla_data` 的前五行使用`head` 函數的資料幀。


In [29]:
tsla_data.reset_index(inplace = True)
tsla_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume


## 使用網頁抓取提取特斯拉收入資料


使用`requests`庫下載網頁https://cf-courses-data.s3.us.cloud-object-storage。appdomain.cloud /IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm 將回應文字儲存為名為`html_data` 的變數。


In [30]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
url = " https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
data = requests.get(url).text

使用`beautiful_soup`解析 html 資料。


In [31]:
html_data =  BeautifulSoup(data, 'html.parser')
print(html_data)


<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue" rel="canonical"/>
<title>Tesla Revenue 2010-2022 | TSLA | MacroTrends</title>
<meta content="Tesla annual/quarterly revenue history and growth rate from 2010 to 2022. Revenue can be defined as the amount of money a company receives from its customers in exchange for the sales of goods or services.  Revenue is the top line item on an income statement from which all costs and expenses are subtracted to arrive at net income.    
				
				&lt;ul style='margin-top:10px;'&gt;
				&lt;li&gt;Tesla revenue for the quarter ending September 30,

使用`BeautifulSoup` 或`read_html` 函數使用`Tesla Revenue` 提取表格並將其儲存到名為。 和 `tesla_revenue`。資料框應包含列 `DateRevenue`


In [32]:
tsla_revenue = pd.DataFrame(columns=['Date', 'Revenue']) 

revenue_data = []
for table in html_data.find_all('table'):
    if 'Tesla Quarterly Revenue' in table.find('th').text:
        rows = table.find_all('tr')
        for row in rows:
            col = row.find_all('td')
            if col != []:
                date = col[0].text
                revenue = col[1].text.replace(',', '').replace('$', '')
                revenue_data.append({"Date": date, "Revenue": revenue})

tsla_revenue = pd.concat([tsla_revenue, pd.DataFrame(revenue_data)], ignore_index=True)

執行下列行從 `Revenue` 欄位中刪除逗號和美元符號。


In [33]:
tsla_revenue["Revenue"] = tsla_revenue["Revenue"].str.replace(",|\$", "", regex=True)

執行下列行以刪除 Revenue 列中的 null 或空字串。


In [34]:
tsla_revenue.dropna(inplace=True)

tsla_revenue = tsla_revenue[tsla_revenue['Revenue'] != ""]

使用 `tail` 函數顯示 `tesla_revenue` 資料幀的最後 5 行。 

In [35]:
tsla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


## 使用yfinance擷取股票資料


使用`Ticker`函數輸入我們想要提取資料以建立股票物件的股票的股票代碼。 該股為 `GameStop`，其股票代號為「GME」。


In [36]:
GameStop = yf.Ticker("GME")

使用股票代碼物件和函數`history`提取股票資訊並將其保存在名為 `gme_data` 的資料框中。 將 `period` 參數設為 `max`，以便我們獲得最長時間的資訊。


In [37]:
gme_data = GameStop.history(period="max")

GME: No data found for this date range, symbol may be delisted


使用 `gme_data` 資料幀上的 `reset_index(inplace=True)` 函數 (**Reset the index**重置索引) ，並使用 `head` 函數顯示 `gme_data` 資料幀的前五行。


In [38]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume


## 使用Webscraping提取GME收入資料


使用`requests`庫下載網頁https://cf-courses-data.s3.us.cloud-object-storage。appdomain.cloud /IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html。將回應文字儲存為名為 `html_data` 的變數。


In [39]:
url1 = " https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
data1 = requests.get(url1).text

使用`beautiful_soup`解析 html 資料。


In [40]:
html_data1 = BeautifulSoup(data1,'html.parser')

使用`BeautifulSoup`或`read_html`函數提取包含`GameStop Revenue`的表並將其儲存到名為`gme_revenue`的資料幀中。 資料框應包含 `Date` 和`Revenue`欄位。 確保使用與問題 2 中類似的方法從`Revenue`列中刪除逗號和美元符號。


In [42]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in html_data1.find_all('table'):

    if ('GameStop Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')
        
        for row in rows:
            col = row.find_all('td')
            
            if col != []:
                date = col[0].text
                revenue = col[1].text.replace(',','').replace('$','')

                new_row = pd.DataFrame({"Date": [date], "Revenue": [revenue]})
                gme_revenue = pd.concat([gme_revenue, new_row], ignore_index=True)

使用 `tail` 函數顯示 `gme_revenue` 資料幀的最後五行。

In [43]:
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


## 繪製 Tesla 股票圖


使用 `make_graph` 函數繪製特斯拉股票資料的圖表，並提供圖表的標題。 呼叫 `make_graph` 函式的結構是 `make_graph(tesla_data, tesla_revenue, 'Tesla')`。 請注意，該圖表僅顯示截至 2021 年 6 月的數據。


In [44]:
make_graph(tsla_data, tsla_revenue, 'Tesla')

## 繪製 GameStop 股票圖


使用`make_graph`函數繪製 GameStop 股票資料的圖表，並提供圖表的標題。呼叫`make_graph`函數的結構是`make_graph(gme_data, gme_revenue, 'GameStop')`。請注意，該圖表僅顯示截至 2021 年 6 月的數據。


In [45]:
make_graph(gme_data, gme_revenue, 'GameStop')